In [1]:
import os
import librosa
import matplotlib.pyplot as plt
import soundfile
import numpy as np
import librosa.display
import glob
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    #here file_name is the full path location of file
    with soundfile.SoundFile(file_name) as sound_file:
        print("Currently running: ", file_name)
        X, sample_rate = librosa.load(file_name)
        if chroma:
            #stft is used to determine the sinusoidal frequency and phase content of local sections of a signal as it changes over time
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        #mfc is a representation of the short-term power spectrum of a sound
        # mfcc collectively make a mfc
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate,n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
    return result

In [3]:
emotions = {'01':'neutral', '02':'calm', '03':'happy', '04':'sad', '05':'angry', '06':'fearful', '07':'disgust', '08':'surprised'}
observed_emotions=['calm', 'happy', 'sad', 'angry']

In [4]:
def load_data(test_size):
    x,y = [],[]
    for file in glob.glob("E:\Machine learning\Machine learning Project\Actor_*\*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
             continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return model_selection.train_test_split(x, y, train_size=1-test_size, test_size=test_size, random_state=101)

In [5]:
X_train = []
X_test = []
y_train = []
y_test = []
X_train, X_test, y_train, y_test = load_data(test_size=0.20)

Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-01-01-01-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-01-01-02-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-01-02-01-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-01-02-02-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-02-01-01-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-02-01-02-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-02-02-01-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-02-02-02-02-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-03-01-01-01-01.wav
Currently running:  E:\Machine learning\Machine learning Project\Actor_01\03-01-03-01-01-02-01.wav
Currently 

In [6]:
print(len(X_train))
print(len(X_test))

614
154


# PCA

In [7]:

# pre-processing the data
# for applying PCA data must be normalised first
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler() 
  
# fit_transform finds mean and standard deviation and then returns the transformed data
X_train_norm = sc.fit_transform(X_train)
# transform only transforms the data with previous required values
X_test_norm = sc.transform(X_test)

print(X_train_norm)

[[-0.82293001  0.89432591  0.71564485 ... -0.07101651 -0.45758372
  -0.22359131]
 [ 0.35891905 -1.23954896 -0.49543797 ... -0.32590528 -0.10573626
  -0.23294874]
 [-1.6820654   0.8094188   2.0017516  ... -1.23899756 -0.70526124
  -0.58666646]
 ...
 [ 1.96949821 -3.12406594 -2.06976256 ...  0.19817993 -0.17371296
   0.49340017]
 [-1.06041136  0.86089555  2.35094504 ... -0.40144092 -0.89088884
  -1.04129509]
 [ 0.35161733  0.85362681  0.06508391 ... -0.53246429 -0.05419891
  -0.07885832]]


In [8]:
# apply PCA
from sklearn.decomposition import PCA 
  
pca = PCA(n_components = 'mle')
  
X_train_pca = pca.fit_transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)
  
explained_variance = pca.explained_variance_ratio_ 
print(explained_variance)

[0.24421205 0.14919659 0.13137659 0.07387014 0.04728523 0.03927546
 0.0341891  0.02797482 0.02505939 0.02335726 0.01974018 0.01764763
 0.01585538 0.01301684 0.01292695 0.01182376 0.01037964 0.01010761
 0.00918408 0.00844163 0.00718018 0.00709679 0.00602028 0.00553482
 0.00514399 0.00502929 0.00442475 0.00417603 0.00371105 0.00349578
 0.00332897 0.00319241 0.00276215 0.00261272 0.00239888]


# Decision Tree without PCA

In [9]:
#Fitting Decision Tree classifier to the training set  
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(X_train, y_train)  
y_pred= classifier.predict(X_test)  
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 61.69%
              precision    recall  f1-score   support

       angry       0.62      0.76      0.68        41
        calm       0.86      0.57      0.68        44
       happy       0.53      0.49      0.51        39
         sad       0.51      0.67      0.58        30

    accuracy                           0.62       154
   macro avg       0.63      0.62      0.61       154
weighted avg       0.64      0.62      0.62       154

[[31  0  8  2]
 [ 3 25  4 12]
 [14  1 19  5]
 [ 2  3  5 20]]


# Decision Tree with PCA

In [10]:
#Fitting Decision Tree classifier to the training set  
from sklearn.tree import DecisionTreeClassifier  
classifier= DecisionTreeClassifier(criterion='entropy', random_state=0)  
classifier.fit(X_train_pca, y_train)  
y_pred= classifier.predict(X_test_pca)  
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 53.90%
              precision    recall  f1-score   support

       angry       0.58      0.61      0.60        41
        calm       0.76      0.66      0.71        44
       happy       0.40      0.44      0.42        39
         sad       0.39      0.40      0.39        30

    accuracy                           0.54       154
   macro avg       0.53      0.53      0.53       154
weighted avg       0.55      0.54      0.54       154

[[25  1 10  5]
 [ 3 29  7  5]
 [ 9  4 17  9]
 [ 6  4  8 12]]


# KNN without PCA

In [11]:
from sklearn.neighbors import KNeighborsClassifier as knn

knn = knn(n_neighbors=4) 
#Train the model
knn.fit(X_train, y_train)
y_pred=knn.predict(X_test)
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 70.13%
              precision    recall  f1-score   support

       angry       0.63      0.88      0.73        41
        calm       0.83      0.86      0.84        44
       happy       0.63      0.56      0.59        39
         sad       0.75      0.40      0.52        30

    accuracy                           0.70       154
   macro avg       0.71      0.68      0.67       154
weighted avg       0.71      0.70      0.69       154

[[36  0  5  0]
 [ 3 38  0  3]
 [15  1 22  1]
 [ 3  7  8 12]]


# KNN with PCA

In [12]:
from sklearn.neighbors import KNeighborsClassifier as knn


knn = knn(n_neighbors=3) 
#Train the model
knn.fit(X_train_pca, y_train)
#DataFlair - Predict for the test set
y_pred=knn.predict(X_test_pca)
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


Accuracy: 73.38%
              precision    recall  f1-score   support

       angry       0.67      0.80      0.73        41
        calm       0.84      0.86      0.85        44
       happy       0.75      0.62      0.68        39
         sad       0.64      0.60      0.62        30

    accuracy                           0.73       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.74      0.73      0.73       154

[[33  1  7  0]
 [ 2 38  0  4]
 [ 9  0 24  6]
 [ 5  6  1 18]]


# SVM without PCA

In [13]:
from sklearn.svm import SVC # "Support vector classifier"  
model_SVM = SVC(kernel='rbf', random_state=0, gamma=0.01, C=3)  
model_SVM.fit(X_train, y_train)
#Predicting the test set result  
y_pred_SVM = model_SVM.predict(X_test)

print ("Accuracy of SVM : ", accuracy_score(y_test, y_pred_SVM))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy of SVM :  0.5909090909090909
              precision    recall  f1-score   support

       angry       0.67      0.80      0.73        41
        calm       0.84      0.86      0.85        44
       happy       0.75      0.62      0.68        39
         sad       0.64      0.60      0.62        30

    accuracy                           0.73       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.74      0.73      0.73       154

[[33  1  7  0]
 [ 2 38  0  4]
 [ 9  0 24  6]
 [ 5  6  1 18]]


# SVM with PCA

In [14]:
from sklearn.svm import SVC # "Support vector classifier"  
model_SVM = SVC(kernel='rbf', random_state=0, gamma=0.01, C=3)  
model_SVM.fit(X_train_pca, y_train)
#Predicting the test set result  
y_pred_SVM = model_SVM.predict(X_test_pca)
print ("Accuracy of SVM : ", accuracy_score(y_test, y_pred_SVM))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy of SVM :  0.7662337662337663
              precision    recall  f1-score   support

       angry       0.67      0.80      0.73        41
        calm       0.84      0.86      0.85        44
       happy       0.75      0.62      0.68        39
         sad       0.64      0.60      0.62        30

    accuracy                           0.73       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.74      0.73      0.73       154

[[33  1  7  0]
 [ 2 38  0  4]
 [ 9  0 24  6]
 [ 5  6  1 18]]


# Random Forest without PCA

In [15]:
from sklearn.ensemble import RandomForestClassifier
randomForest = RandomForestClassifier(max_depth=20, random_state=0)
randomForest.fit(X_train, y_train)
y_pred_forest = randomForest.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, y_pred_forest))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy:  0.7337662337662337
              precision    recall  f1-score   support

       angry       0.67      0.80      0.73        41
        calm       0.84      0.86      0.85        44
       happy       0.75      0.62      0.68        39
         sad       0.64      0.60      0.62        30

    accuracy                           0.73       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.74      0.73      0.73       154

[[33  1  7  0]
 [ 2 38  0  4]
 [ 9  0 24  6]
 [ 5  6  1 18]]


# Random Forest with PCA


In [16]:
randomForest.fit(X_train_pca, y_train)
y_pred_forest = randomForest.predict(X_test_pca)
print("Accuracy: ", accuracy_score(y_test, y_pred_forest))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy:  0.8051948051948052
              precision    recall  f1-score   support

       angry       0.67      0.80      0.73        41
        calm       0.84      0.86      0.85        44
       happy       0.75      0.62      0.68        39
         sad       0.64      0.60      0.62        30

    accuracy                           0.73       154
   macro avg       0.73      0.72      0.72       154
weighted avg       0.74      0.73      0.73       154

[[33  1  7  0]
 [ 2 38  0  4]
 [ 9  0 24  6]
 [ 5  6  1 18]]


# MLP without PCA


In [17]:

from sklearn.neural_network import MLPClassifier

model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
#Train the model
model.fit(X_train,y_train)
#DataFlair - Predict for the test set
y_pred=model.predict(X_test)
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 64.94%
              precision    recall  f1-score   support

       angry       0.79      0.54      0.64        41
        calm       0.80      0.80      0.80        44
       happy       0.61      0.72      0.66        39
         sad       0.42      0.50      0.45        30

    accuracy                           0.65       154
   macro avg       0.65      0.64      0.64       154
weighted avg       0.67      0.65      0.65       154

[[22  1 12  6]
 [ 1 35  0  8]
 [ 2  2 28  7]
 [ 3  6  6 15]]


# MLP with PCA


In [18]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
#Train the model
model.fit(X_train_pca,y_train)
#DataFlair - Predict for the test set
y_pred=model.predict(X_test_pca)
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: {:.2f}%".format(accuracy*100))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 81.82%
              precision    recall  f1-score   support

       angry       0.88      0.93      0.90        41
        calm       0.87      0.91      0.89        44
       happy       0.78      0.74      0.76        39
         sad       0.68      0.63      0.66        30

    accuracy                           0.82       154
   macro avg       0.80      0.80      0.80       154
weighted avg       0.81      0.82      0.82       154

[[38  0  3  0]
 [ 0 40  0  4]
 [ 4  1 29  5]
 [ 1  5  5 19]]


# Ensemble Method using Voting Classifier

In [19]:
from sklearn.ensemble import VotingClassifier 
estimator = [] 
estimator.append(('MLPC',MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
)) 
estimator.append(('RFC',RandomForestClassifier(max_depth=20, random_state=0))) 
estimator.append(('SVC',SVC(kernel='rbf',probability = True, random_state=0, gamma=0.01, C=3)  
)) 
vot_hard = VotingClassifier(estimators = estimator, voting ='hard') 
vot_hard.fit(X_train_pca, y_train) 
y_pred = vot_hard.predict(X_test_pca) 
  
# using accuracy_score metric to predict accuracy 
score = accuracy_score(y_test, y_pred) 
print("Accuracy Score with hard voting ",score) 
  
# Voting Classifier with soft voting 
vot_soft = VotingClassifier(estimators = estimator, voting ='soft') 
vot_soft.fit(X_train_pca, y_train) 
y_pred = vot_soft.predict(X_test_pca) 
  
# using accuracy_score 
score = accuracy_score(y_test, y_pred) 
print("Accuracy Score with soft voting ",score) 

Accuracy Score with hard voting  0.8441558441558441
Accuracy Score with soft voting  0.8181818181818182
